In [1]:
# Import dependencies
import numpy as np
import pandas as pd

In [2]:
# Read our schools data into pandas
csv_path1 = "data/schools_complete.csv"
schools_df = pd.read_csv(csv_path1)

In [3]:
# Read the students data into pandas
csv_path2 = "data/students_complete.csv"
students_df = pd.read_csv(csv_path2)

In [4]:
# Find the total number of schools
total_schools = schools_df['School ID'].count()

In [5]:
# Sum down the columns to reach total students and total budget
total_students = schools_df['size'].sum()
total_budget = schools_df['budget'].sum()

In [6]:
# Calculate the average Math and Reading scores
average_math_score = (students_df['math_score'].sum())/(total_students)
average_reading_score = (students_df['reading_score'].sum())/(total_students)

In [7]:
# Calculate the percent passing math and percent passing reading scores
passing_math = students_df.loc[(students_df['math_score'] >= 70)]
number_passing_math = passing_math['math_score'].count()
percent_passing_math = (number_passing_math/total_students)*100

passing_reading = students_df.loc[(students_df['reading_score'] >= 70)]
number_passing_reading = passing_reading['reading_score'].count()
percent_passing_reading = (number_passing_reading/total_students)*100

In [8]:
# Calculate the Overall Passing Rate as the average of the above two
overall_passing = (percent_passing_math + percent_passing_reading)/2

In [9]:
# Create the District Summary Table
district_summary = pd.DataFrame({"Total Schools": [total_schools], "Total Students": [total_students], 
                                    "Total Budget": [total_budget], "Average Math Score": [average_math_score], 
                                    "Average Reading Score":[average_reading_score], "% Passing Math": [percent_passing_math],
                                   "% Passing Reading": [percent_passing_reading], "Overall Passing Rate": [overall_passing]})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


TABLE: District Summary. Shows key statistics for the district.

ANALYSIS: The district as a whole, if resources are available, needs to invest more in its students. While some of the charter
schools are doing well as is seen in the other tables, the overall passing rate for the district is only incrementally over
80%, and the percentage passing math is at 75%. Clearly more must be done for the students.

In [10]:
# Now, produce the School Summary table. First, find the average math and reading scores
sorted_students_df = students_df.groupby("school")
average_math_school = sorted_students_df["math_score"].mean().to_frame()
average_reading_school = sorted_students_df["reading_score"].mean().to_frame()

In [11]:
# Calculate the number passing math and number passing reading. From this, we will find out the percent passing each
math_passing = students_df[students_df["math_score"] > 70].groupby('school').count()["math_score"].to_frame()
reading_passing = students_df[students_df["reading_score"] > 70].groupby('school').count()["reading_score"].to_frame()

In [12]:
# Count the total number of students per school and save as DataFrame
students_in_school = sorted_students_df["school"].count().to_frame()

In [13]:
# Create a table that lists all of the scores for each school and save as DataFrame
from functools import reduce

mergeVars = [average_math_school, average_reading_school, math_passing, reading_passing, students_in_school]
scores_school = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True), mergeVars)
scores_school.columns = ["Average Math Score", "Average Reading Score", "n Passed Math", "n Passed Reading", "# Students"]
scores_school["% Passing Math"] = (scores_school["n Passed Math"] / scores_school["# Students"] * 100)
scores_school["% Passing Reading"] = (scores_school["n Passed Reading"] / scores_school["# Students"] * 100)
scores_school["Overall Passing Rate"] = (scores_school["% Passing Math"]+scores_school["% Passing Reading"])/2
scores_school["Total Students"] = scores_school["# Students"]
scores_school = scores_school[["Total Students", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]

In [14]:
# Now create an overview table summarizing key statistics about each school. First calculate the Per Student Budget
schools_df["Per student budget"] = schools_df['budget']/schools_df['size']

In [15]:
# Rename the columns in the schools DataFrame
schools_df = schools_df.rename(columns = {"name": "school", "type": "School Type", "size": "Total Students", "budget": "Total budget"})

In [16]:
# Merge the two DataFrames(schools data frame and school scores data frame) into one table
school_summary = pd.merge(schools_df, scores_school, on = "Total Students", how = "outer")

In [17]:
# Finalize our summary table that includes all of the information from each school
school_summary

,School ID,school,School Type,Total Students,Total budget,Per student budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,71.066164
1,1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,71.091896
2,2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,89.892107,92.617831,91.254969
3,3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,71.467098
4,4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,89.713896,93.392371,91.553134
5,5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,90.932983,93.254490,92.093736
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,89.558665,93.864370,91.711518
7,7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,64.630225,79.300643,71.965434
8,8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,91.686183
9,9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,91.943867


TABLE: School Summary. This table summarizes the key details and statistics for each school.

ANALYSIS: Note that the district schools as a group are larger, have a higher per student budget, and have lower scores and passing rates. This is a general trend.

In [18]:
# Now, create a table of the five top performing schools based on Overall Passing Rate

topSchools = school_summary.sort_values("Overall Passing Rate", ascending = False).reset_index()
topSchools = topSchools.loc[0:4,:]
topSchools

,index,School ID,school,School Type,Total Students,Total budget,Per student budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,5,5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,90.932983,93.254490,92.093736
1,9,9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,91.943867
2,10,10,Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,90.277778,93.444444,91.861111
3,6,6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,89.558665,93.864370,91.711518
4,8,8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,91.686183


TABLE: Top Performing Schools. This table summarizes the key details and statistics for the five schools with the highest overall passing rate.

ANALYSIS: All of the top five performing schools are Charter schools.

In [19]:
# Now, create a table of the bottom five schools based on the same

bottomSchools = school_summary.sort_values("Overall Passing Rate", ascending = True).reset_index()
bottomSchools = bottomSchools.loc[0:4,:]
bottomSchools

,index,School ID,school,School Type,Total Students,Total budget,Per student budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,11,11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,64.066017,77.744436,70.905226
1,0,0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,71.066164
2,12,12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,63.852132,78.281874,71.067003
3,1,1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,71.091896
4,3,3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,71.467098


TABLE: Bottom Performing Schools. This table summarizes the key details and statistics for the five schools with the lowest overall passing rate.

ANALYSIS: All of the bottom five performing schools are Charter schools. This sends an obvious message about the disparity between the quality of education in district versus charter schools.

In [20]:
# Now, work on the Math and Reading Scores by Grade. First, sort students by grade as how in the School Summary problem they are sorted by school
sorted_students_df = students_df.groupby("school")

In [21]:
# Math Scores by Grades

# 9th grade
math9 = students_df[students_df["grade"] == "9th"].groupby('school')["math_score"].mean().to_frame()
math9 = math9.rename(columns = {"math_score": "9th"})

# 10th grade
math10 = students_df[students_df["grade"] == "10th"].groupby('school')["math_score"].mean().to_frame()
math10 = math10.rename(columns = {"math_score": "10th"})

# 11th grade
math11 = students_df[students_df["grade"] == "11th"].groupby('school')["math_score"].mean().to_frame()
math11 = math11.rename(columns = {"math_score": "11th"})

# 12th grade
math12 = students_df[students_df["grade"] == "12th"].groupby('school')["math_score"].mean().to_frame()
math12 = math12.rename(columns = {"math_score": "12th"})

In [22]:
# Merge grade dfs together
from functools import reduce

mergemath = [math9, math10, math11, math12]
math_Grades = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True), mergemath)
math_Grades.reset_index()

,school,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


TABLE: Math Scores By Grade. This table shows the average math scores for students in grades 9, 10, 11, and 12 by school.

ANALYSIS: The scores across grades are pretty consistent. Schools tend to score either in the 77% area or the 83% area with little variation among grades. This performance still leaves much to be desired, however.

In [23]:
# Reading scores by grades

# 9th grade
reading9 = students_df[students_df["grade"] == "9th"].groupby('school')["reading_score"].mean().to_frame()
reading9 = reading9.rename(columns = {"reading_score": "9th"})

# 10th grade
reading10 = students_df[students_df["grade"] == "10th"].groupby('school')["reading_score"].mean().to_frame()
reading10 = reading10.rename(columns = {"reading_score": "10th"})

# 11th grade
reading11 = students_df[students_df["grade"] == "11th"].groupby('school')["reading_score"].mean().to_frame()
reading11 = reading11.rename(columns = {"reading_score": "11th"})

# 12th grade
reading12 = students_df[students_df["grade"] == "12th"].groupby('school')["reading_score"].mean().to_frame()
reading12 = reading12.rename(columns = {"reading_score": "12th"})

In [24]:
# Merge grade dfs together
from functools import reduce

mergereading = [reading9, reading10, reading11, reading12]
reading_Grades = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True), mergereading)
reading_Grades.reset_index()

,school,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


TABLE: Reading Scores By Grade. This table shows the average reading scores for students in grades 9, 10, 11, and 12 by school.

ANALYSIS: When scanning the math and reading scores by grade tables, there is a clear difference between math and reading scores: reading scores tend to be 4 to 5 percentage points higher than math scores. This difference suggests that an improvement in curriculum should be made. More time or resources should be spent on math so that students score equally well in both subjects.

In [25]:
# Now, work on creating a data frame for scores by school spending
# Create the basic skeleton
school_budget = school_summary[['Per student budget',
                             'Average Math Score',
                             'Average Reading Score',
                             '% Passing Math',
                             '% Passing Reading',
                             'Overall Passing Rate']]       \
                .reset_index()

In [26]:
# Create reasonable bins with which to group school spending into
bins = [575, 595, 615, 635, 655]

# Create labels for these bins
group_labels = ["575 to 595", "595 to 615", "615 to 635", "635 to 655"]

In [27]:
# Slice the data and place it into bins
school_budget["Per student budget group"] = pd.cut(school_budget["Per student budget"], bins, labels=group_labels)

In [28]:
# Final commands to generate a scores by spending report
school_budget = school_budget.groupby("Per student budget group")
print(school_budget["index"].count())
school_budget[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].mean()

Per student budget group
575 to 595    4
595 to 615    2
615 to 635    2
635 to 655    7
Name: index, dtype: int64


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per student budget group,,,,,
575 to 595,83.455399,83.933814,90.350436,93.325838,91.838137
595 to 615,83.599686,83.885211,90.788049,92.410786,91.599418
615 to 635,80.199966,82.425360,77.172061,86.346507,81.759284
635 to 655,77.866721,81.368774,67.957362,80.268067,74.112715


TABLE: Scores by School Spending. This table shows the passing rate for different school spending groups.

ANALYSIS: Surprisingly, the overall passing rate is lowest for the students in the highest per student budget group. However, we will also see that the district schools have a much lower passing rate than the charter schools - so it seems that district schools spend more resources per student and are also less efficient (ie. perform worse).

In [29]:
# Now, we want to create a DataFrame conveying the school scores by size of school.
# Repeat the process of the above section, but include Total Students rather than Per student budget:
school_size = school_summary[['Total Students',
                             'Average Math Score',
                             'Average Reading Score',
                             '% Passing Math',
                             '% Passing Reading',
                             'Overall Passing Rate']]       \
                .reset_index()

In [30]:
# Create reasonable bins with which to group school spending into
bins2 = [400, 2000, 3600, 5200]

# Create labels for these bins
group_labels2 = ["Small", "Medium", "Large"]

In [31]:
# Slice the data and place it into bins
pd.cut(school_size["Total Students"], bins2, labels=group_labels2)
school_size["School Size"] = pd.cut(school_size["Total Students"], bins2, labels=group_labels2)

In [32]:
# Finalize a scores by school size report
school_size = school_size.groupby("School Size")
print(school_size["index"].count())
school_size[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                "Overall Passing Rate"]].mean()

School Size
Small     7
Medium    4
Large     4
Name: index, dtype: int64


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small,83.502373,83.883125,90.281832,93.024001,91.652916
Medium,78.429493,81.769122,70.938952,82.002937,76.470945
Large,77.063340,80.919864,64.323717,78.378664,71.351190


TABLE: Scores by School Size. This table shows the scores and passing rates for schools of different sizes.

ANALYSIS: Here we see that the larger schools perform the worst and the medium schools also perform poorly. The small schools, which happen to be most of the charter schools, have the highest overall passing rate.

In [33]:
# Now, we are producing a similar table based on school type (district or charter).

school_type = school_summary[['School Type',
                             'Average Math Score',
                             'Average Reading Score',
                             '% Passing Math',
                             '% Passing Reading',
                             'Overall Passing Rate']]       \
                .reset_index()

In [34]:
# Produce the final table.
school_type = school_type.groupby("School Type")
school_type[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                "Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543


TABLE: Scores by School Type. This scores and passing rates for district versus charter schools.

ANALYSIS: This is the most striking analysis table. There is a huge difference in performance between the district and charter schools. It is clear that the administration needs to look into and improve the district schools' performance.